In [1]:
import kagglehub

In [2]:
d2 = kagglehub.dataset_download("benroshan/online-food-delivery-preferencesbangalore-region")

print("Path to dataset files:", d2)

100%|██████████| 24.0k/24.0k [00:00<00:00, 3.73MB/s]

Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/benroshan/online-food-delivery-preferencesbangalore-region/versions/3


In [3]:
d5 = kagglehub.dataset_download("saurabhbadole/zomato-delivery-operations-analytics-dataset")

print("Path to dataset files:", d5)

100%|██████████| 1.43M/1.43M [00:00<00:00, 38.2MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/saurabhbadole/zomato-delivery-operations-analytics-dataset/versions/1


Cleaning Dataset D5

In [4]:
import pandas as pd
df_5 = pd.read_csv("/root/.cache/kagglehub/datasets/saurabhbadole/zomato-delivery-operations-analytics-dataset/versions/1/Zomato Dataset.csv")

In [5]:
df_5.count()

,0
ID,45584
Delivery_person_ID,45584
Delivery_person_Age,43730
Delivery_person_Ratings,43676
Restaurant_latitude,45584
Restaurant_longitude,45584
Delivery_location_latitude,45584
Delivery_location_longitude,45584
Order_Date,45584
Time_Orderd,43853


For column Time_Orderd, remove Nan, 0.x and 1 values:

In [6]:
df_5 = df_5[~df_5['Time_Orderd'].apply(lambda x: isinstance(x, float) or '0.' in str(x) or str(x) == '1')]


For column Time_Order_picked, remove 0.x and 1 values:

In [7]:
df_5 = df_5[~df_5['Time_Order_picked'].apply(lambda x: '0.' in str(x) or str(x) == '1' or '24:' in str(x))]


Cleaning the Dataset which had nan values.

In [8]:
df_5_cleaned = df_5.dropna()
df_5_cleaned.count()

,0
ID,33402
Delivery_person_ID,33402
Delivery_person_Age,33402
Delivery_person_Ratings,33402
Restaurant_latitude,33402
Restaurant_longitude,33402
Delivery_location_latitude,33402
Delivery_location_longitude,33402
Order_Date,33402
Time_Orderd,33402


D5 is now cleaned!

Now create a new dataset by simulation:

Generate 34326 values of date and time:

In [9]:
num_rows = 34326

In [10]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
# Define the start and end dates
start_date = "2022-04-01"
end_date = "2024-03-31"
# Create a range of dates
date_range = pd.date_range(start=start_date, end=end_date)
# Sample random dates from the range
random_dates = np.random.choice(date_range, size=num_rows, replace=True)
# Create a DataFrame with the random dates
df_5_new = pd.DataFrame(random_dates, columns=['Date'])
# Add a random time component to each date
df_5_new['Date'] = df_5_new['Date'] + pd.to_timedelta(np.random.randint(0, 24 * 60 * 60, size=len(df_5_new)), unit='s')
print(df_5_new.head())
# Optionally, save it to a CSV file
df_5_new.to_csv("generated_dates.csv", index=False)

                 Date
0 2023-02-20 18:26:34
1 2024-01-21 07:17:11
2 2023-06-12 07:38:10
3 2023-12-10 15:26:28
4 2024-01-11 21:21:22


Split above generated data into 2 new columns Order_Date and Time_Orderd:

In [11]:
import pandas as pd

# Assuming 'Date' column is already a datetime type

# Extract date and time components using vectorized operations
df_5_new['Order_Date'] = df_5_new['Date'].dt.strftime("%d-%m-%Y")
df_5_new['Time_Orderd'] = df_5_new['Date'].dt.strftime("%H:%M")

# Save to CSV
df_5_new.to_csv("generated_dates.csv", index=False)

Drop existing column Date as it is not needed now:

In [12]:
df_5_new = df_5_new.drop('Date', axis=1)

In [13]:
df_5_new.to_csv("generated_dates.csv", index=False)

Generate new column Time_Order_picked based on Time_Orderd column:

In [14]:
def add_random_minutes(time_str):
    # Convert 'time_ordered' to a datetime object
    time_obj = pd.to_datetime(time_str, format='%H:%M')

    # Generate a random value between 5 and 30 (minutes)
    random_minutes = np.random.randint(5, 31)

    # Add the random minutes to the time
    new_time_obj = time_obj + pd.Timedelta(minutes=random_minutes)

    if new_time_obj.hour == 24:
        new_time_obj = new_time_obj.replace(hour=0, minute=new_time_obj.minute)

    # Return the new time as a string in HH:MM format
    return new_time_obj.strftime('%H:%M')

# Create a new column with the random added minutes
df_5_new['Time_Order_picked'] = df_5_new['Time_Orderd'].apply(add_random_minutes)

df_5_new.to_csv("generated_dates.csv", index=False)

Generate new column Weather_conditions:

In [15]:

# List of possible weather conditions
weather_conditions = ['Windy', 'Fog', 'Storm', 'Sandstorm', 'Rainy', 'Cloudy']

# Randomly assign weather conditions to the 'weather' column
df_5_new['Weather_conditions'] = np.random.choice(weather_conditions, size = num_rows)

df_5_new.to_csv("generated_dates.csv", index=False)


In [16]:
df_5_new.head()

,Order_Date,Time_Orderd,Time_Order_picked,Weather_conditions
0,20-02-2023,18:26,18:44,Windy
1,21-01-2024,07:17,07:38,Windy
2,12-06-2023,07:38,07:46,Windy
3,10-12-2023,15:26,15:43,Storm
4,11-01-2024,21:21,21:44,Cloudy


We have now generated a new dataset with required values!

Now we have to merge df_5 and df_5_new:

In [17]:
final_df_5 = pd.DataFrame()

df_5_columns = ['Order_Date',	'Time_Orderd',	'Time_Order_picked',	'Weather_conditions']
final_df_5_columns = ['Order_Date',	'Time_Orderd',	'Time_Order_picked',	'Weather_conditions']

final_df_5[final_df_5_columns] = df_5_cleaned[df_5_columns].copy()

In [18]:
final_df_5 = pd.concat([final_df_5, df_5_new], ignore_index=True)

Shuffle final dataset and save to csv

In [19]:
# Shuffle the rows
final_df_5 = final_df_5.sample(frac=1, random_state=42)  # frac=1 means sample all rows

In [20]:
final_df_5.to_csv("D5.csv", index=False)

Dataset D2

In [21]:
import pandas as pd
df_2 = pd.read_csv("/root/.cache/kagglehub/datasets/benroshan/online-food-delivery-preferencesbangalore-region/versions/3/onlinedeliverydata.csv")

To remove NaN values in D2

In [22]:
df_2 = df_2.dropna()

In [23]:
val = ['Zomato', 'Swiggy']
for i in df_2['Medium (P1)']:
  if str(i) == "Food delivery apps":
    df_2['Medium (P1)'] = np.random.choice(val, size=len(df_2))

In [24]:
final_df_2 = pd.DataFrame()

df_2_columns = ['Age', 'Gender', 'Marital Status', 'Occupation', 'Monthly Income',	'Medium (P1)', 'Meal(P1)', 'Influence of rating', 'Freshness ', 'Temperature', 'Good Taste ', 'Good Quantity', 'Good Food quality', 'Poor Hygiene', 'Bad past experience', 'High Quality of package', 'Late Delivery', 'Long delivery time',	'Delay of delivery person getting assigned', 'Delay of delivery person picking up food', 'Maximum wait time', 'Less Delivery time', 'Wrong order delivered',	'Missing item', 'Order placed by mistake']

final_df_2_columns = ['Age', 'Gender', 'Marital_Status', 'Occupation', 'Monthly_Income',	'Order_Preference', 'Meal_Type', 'Influence_of_Rating', 'Freshness_of_Food', 'Temperature_of_Food', 'Taste_of_Food', 'Quantity_of_Food', 'Food_Quality', 'Poor_Hygiene', 'Bad_Past_Experience', 'High_Quality_of_Package', 'Late_Delivery', 'Long_Delivery_Time',	'Delay_of_Delivery_Person_Getting_Assigned', 'Delay_of_Delivery_Person_Picking_Up_Food', 'Maximum_Wait_Time', 'Less_Delivery_Time', 'Wrong_Order_Delivered',	'Missing_Item', 'Order_Placed_by_Mistake' ]

final_df_2[final_df_2_columns] = df_2[df_2_columns].copy()

In [25]:
final_df_2.to_csv("D2.csv", index=False)

In [26]:
# Number of rows
num_rows = 67341

Simulate columns with values Agree, Stringly Agree and so on

In [27]:
import numpy as np

df_2_new = pd.DataFrame()


# List of possible values
val = ['Agree', 'Strongly Agree', 'Neutral', 'Disagree',  'Strongly Disagree']

# Randomly assign values to the required columns
df_2_new['Food_Quality'] = np.random.choice(val, size=num_rows)
df_2_new['Poor_Hygiene'] = np.random.choice(val, size=num_rows)
df_2_new['Bad_Past_Experience'] = np.random.choice(val, size=num_rows)
df_2_new['Late_Delivery'] = np.random.choice(val, size=num_rows)
df_2_new['Long_Delivery_Time'] = np.random.choice(val, size=num_rows)
df_2_new['Delay_of_Delivery_Person_Getting_Assigned'] = np.random.choice(val, size=num_rows)
df_2_new['Delay_of_Delivery_Person_Picking_Up_Food'] = np.random.choice(val, size=num_rows)
df_2_new['Wrong_Order_Delivered'] = np.random.choice(val, size=num_rows)
df_2_new['Missing_Item'] = np.random.choice(val, size=num_rows)
df_2_new['Order_Placed_by_Mistake'] = np.random.choice(val, size=num_rows)

df_2_new.to_csv("generated_df2.csv", index=False)


Simulate columns with values Important, Unimportant and so on

In [28]:

# List of possible values
val = ['Important', 'Very Important', 'Slightly Important', 'Moderately Important', 'Unimportant']
# Randomly assign values to the required columns
df_2_new['Freshness_of_Food'] = np.random.choice(val, size=num_rows)
df_2_new['Temperature_of_Food'] = np.random.choice(val, size=num_rows)
df_2_new['Taste_of_Food'] = np.random.choice(val, size=num_rows)
df_2_new['Quantity_of_Food'] = np.random.choice(val, size=num_rows)
df_2_new['High_Quality_of_Package'] = np.random.choice(val, size=num_rows)
df_2_new['Less_Delivery_Time'] = np.random.choice(val, size=num_rows)

df_2_new.to_csv("generated_df2.csv", index=False)


Simulate columns with values yes, no and maybe

In [29]:

# List of possible values
val = ['Yes', 'No', 'Maybe']

# Randomly assign values to the required columns
df_2_new['Influence_of_Rating'] = np.random.choice(val, size=num_rows)

df_2_new.to_csv("generated_df2.csv", index=False)


Simulate values of Meal_Type

In [30]:

# List of possible values
val = ['Breakfast', 'Lunch', 'Dinner']

# Randomly assign values to the required columns
df_2_new['Meal_Type'] = np.random.choice(val, size=num_rows)

df_2_new.to_csv("generated_df2.csv", index=False)


Simulate values of Maximum_Wait_Time

In [31]:

# List of possible values
val = ['15 minutes', '30 minutes', '45 minutes', '60 minutes', 'More than 60 minutes']

# Randomly assign values to the required columns
df_2_new['Maximum_Wait_Time'] = np.random.choice(val, size=num_rows)

df_2_new.to_csv("generated_df2.csv", index=False)


Simulate values of order preference

In [32]:
# List of possible values
val = ['Direct Call', 'Website', 'Zomato', 'Swiggy']

dominant_val = ['Zomato', 'Swiggy']
other_val = ['Direct Call', 'Website']

dominant_count = int(0.85 * num_rows)
other_count = int(num_rows - dominant_count)

# Generate choices for dominant values
dominant_choices = np.random.choice(dominant_val, size=dominant_count)

# Generate choices for other values
other_choices = np.random.choice(other_val, size=other_count)

# Concatenate the choices to create the final column
all_choices = np.concatenate([dominant_choices, other_choices])

# Assign the choices to the 'Order_Preference' column
df_2_new['Order_Preference'] = all_choices

# df_2_new['Order_Preference'] = np.random.choice(dominant_val, size=dominant_count)
# df_2_new['Order_Preference'] = np.random.choice(other_val, size=other_count)


df_2_new.to_csv("generated_df2.csv", index=False)

In [33]:
# List of possible values
val_1 = [ 'Male','Female']
# Randomly assign values to the required columns
df_2_new['Age'] = np.random.randint(18, 60, size=num_rows)
df_2_new['Gender'] = np.random.choice(val_1, size=num_rows)
for _, row in df_2_new.iterrows():
    age, gender = row['Age'], row['Gender']

    # Default values for occupation, marital status, and income
    if 18 <= age <= 24:  # Age range for Students
        occupation, marital_status, monthly_income = "Student", "Single", np.random.choice(["No Income", "Below 10000"])
    elif gender == "Female" and age >= 25:  # House wife condition
        occupation = np.random.choice(["House wife", "Employee", "Self Employeed"])
        marital_status, monthly_income = (np.random.choice(["Single", "Married"]), "No Income") if occupation == "House wife" else (np.random.choice(["Single", "Married"]), np.random.choice(["More than 50000", "25001 to 50000", "10001 to 25000", "Below 10000"]))
    else:  # Employees and Self Employeed for other cases
        occupation, marital_status, monthly_income = np.random.choice(["Employee", "Self Employeed"]), np.random.choice(["Single", "Married"]), np.random.choice(["More than 50000", "25001 to 50000", "10001 to 25000", "Below 10000"])

    # Assign values to the row
    df_2_new.at[_, 'Occupation'], df_2_new.at[_, 'Marital_Status'], df_2_new.at[_, 'Monthly_Income'] = occupation, marital_status, monthly_income
df_2_new.to_csv("generated_df2.csv", index=False)

Simulate customer details

In [34]:
final_df_2_new = pd.DataFrame()

df_2_new_columns = ['Age', 'Gender', 'Marital_Status', 'Occupation', 'Monthly_Income',	'Order_Preference', 'Meal_Type', 'Influence_of_Rating', 'Freshness_of_Food', 'Temperature_of_Food', 'Taste_of_Food', 'Quantity_of_Food', 'Food_Quality', 'Poor_Hygiene', 'Bad_Past_Experience', 'High_Quality_of_Package', 'Late_Delivery', 'Long_Delivery_Time',	'Delay_of_Delivery_Person_Getting_Assigned', 'Delay_of_Delivery_Person_Picking_Up_Food', 'Maximum_Wait_Time', 'Less_Delivery_Time', 'Wrong_Order_Delivered',	'Missing_Item', 'Order_Placed_by_Mistake' ]

final_df_2_new_columns = ['Age', 'Gender', 'Marital_Status', 'Occupation', 'Monthly_Income',	'Order_Preference', 'Meal_Type', 'Influence_of_Rating', 'Freshness_of_Food', 'Temperature_of_Food', 'Taste_of_Food', 'Quantity_of_Food', 'Food_Quality', 'Poor_Hygiene', 'Bad_Past_Experience', 'High_Quality_of_Package', 'Late_Delivery', 'Long_Delivery_Time',	'Delay_of_Delivery_Person_Getting_Assigned', 'Delay_of_Delivery_Person_Picking_Up_Food', 'Maximum_Wait_Time', 'Less_Delivery_Time', 'Wrong_Order_Delivered',	'Missing_Item', 'Order_Placed_by_Mistake' ]

final_df_2_new[final_df_2_new_columns] = df_2_new[df_2_new_columns].copy()

Merge generated dataset and original dataset

In [35]:
final_df_2 = pd.concat([final_df_2, final_df_2_new], ignore_index=True)
final_df_2

,Age,Gender,Marital_Status,Occupation,Monthly_Income,Order_Preference,Meal_Type,Influence_of_Rating,Freshness_of_Food,Temperature_of_Food,...,High_Quality_of_Package,Late_Delivery,Long_Delivery_Time,Delay_of_Delivery_Person_Getting_Assigned,Delay_of_Delivery_Person_Picking_Up_Food,Maximum_Wait_Time,Less_Delivery_Time,Wrong_Order_Delivered,Missing_Item,Order_Placed_by_Mistake
0,20,Female,Single,Student,No Income,Zomato,Breakfast,Yes,Moderately Important,Moderately Important,...,Moderately Important,Neutral,Agree,Agree,Agree,30 minutes,Moderately Important,Agree,Agree,Agree
1,24,Female,Single,Student,Below Rs.10000,Swiggy,Snacks,Yes,Very Important,Very Important,...,Very Important,Agree,Strongly agree,Strongly agree,Strongly agree,30 minutes,Very Important,Strongly agree,Strongly agree,Strongly agree
2,22,Male,Single,Student,Below Rs.10000,Zomato,Lunch,Yes,Very Important,Important,...,Very Important,Neutral,Agree,Agree,Agree,45 minutes,Important,Strongly agree,Agree,Neutral
3,22,Female,Single,Student,No Income,Zomato,Snacks,Yes,Very Important,Very Important,...,Important,Neutral,Agree,Agree,Agree,30 minutes,Very Important,Disagree,Disagree,Neutral
4,22,Male,Single,Student,Below Rs.10000,Swiggy,Lunch,Yes,Important,Important,...,Important,Strongly agree,Strongly agree,Strongly agree,Neutral,30 minutes,Important,Neutral,Neutral,Disagree
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67723,30,Female,Married,House wife,No Income,Website,Breakfast,Yes,Very Important,Slightly Important,...,Very Important,Strongly Disagree,Strongly Disagree,Disagree,Strongly Disagree,15 minutes,Very Important,Disagree,Agree,Strongly Agree
67724,43,Female,Single,Self Employeed,More than 50000,Direct Call,Lunch,Yes,Moderately Important,Important,...,Moderately Important,Agree,Strongly Agree,Neutral,Neutral,45 minutes,Moderately Important,Agree,Agree,Strongly Agree
67725,29,Male,Single,Employee,More than 50000,Website,Dinner,No,Important,Slightly Important,...,Slightly Important,Strongly Agree,Strongly Agree,Strongly Agree,Agree,45 minutes,Very Important,Agree,Agree,Strongly Disagree
67726,29,Male,Single,Employee,Below 10000,Website,Dinner,Yes,Unimportant,Very Important,...,Unimportant,Strongly Disagree,Neutral,Strongly Disagree,Disagree,15 minutes,Unimportant,Strongly Agree,Disagree,Agree


In [36]:
final_df_2['Maximum_Wait_Time'] = final_df_2['Maximum_Wait_Time'].apply(lambda x: 60 if "More than 60 minutes" in x else int(x.split()[0]))

Shuffle final dataset and save to csv

In [37]:
final_df_2 = final_df_2.sample(frac=1, random_state=42)  # frac=1 means sample all rows

In [38]:
final_df_2.to_csv("D2.csv", index=False)

Finally get dataset D2 + D5:

In [39]:
dataset_merged = pd.concat([final_df_2, final_df_5], axis=1)

In [40]:
dataset_merged.to_csv("Dataset.csv", index=False)